### 1️⃣ Importaciones y configuración

In [2]:
import sys
import os

BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", ".."))
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

TRAIN_CSV = os.path.join(OUTPUT_DIR, "train_processed.csv")
VAL_CSV   = os.path.join(OUTPUT_DIR, "val_processed.csv")
TEST_CSV  = os.path.join(OUTPUT_DIR, "test_processed.csv")

sys.path.insert(0, BASE_DIR)

from src.part_2.main import run_training
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd


### 2️⃣ Funciones de evaluación de métricas

In [3]:
def mape(y_true, y_pred):
    """Mean Absolute Percentage Error"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    return {
        "MSE": float(mse),
        "MAE": float(mae),
        "MAPE": float(mape_val)
    }


### 4️⃣ Cargar datos usando las variables del notebook


In [ ]:
import pandas as pd

hyperparameter_sets = [
    {"activation": "relu",    "layers": [277, 32, 16, 1],    "epochs": 150, "lr": 0.001,  "momentum": 0.9},
    {"activation": "relu",    "layers": [277, 64, 32, 1],    "epochs": 200, "lr": 0.0005, "momentum": 0.9},
    {"activation": "tanh",    "layers": [277, 20, 10, 1],    "epochs": 200, "lr": 0.05,   "momentum": 0.9},
    {"activation": "tanh",    "layers": [277, 40, 20, 1],    "epochs": 250, "lr": 0.02,   "momentum": 0.8},
    {"activation": "sigmoid", "layers": [277, 20, 10, 1],    "epochs": 70,  "lr": 0.001,  "momentum": 0.9},
    {"activation": "sigmoid", "layers": [277, 50, 25, 1],    "epochs": 120, "lr": 0.005,  "momentum": 0.7},
    {"activation": "linear",  "layers": [277, 1],            "epochs": 200, "lr": 1e-6,   "momentum": 0},
    {"activation": "relu",    "layers": [277, 128, 64, 1],   "epochs": 250, "lr": 0.0003, "momentum": 0.9},
    {"activation": "tanh",    "layers": [277, 64, 32, 1],    "epochs": 150, "lr": 0.01,   "momentum": 0.7},
    {"activation": "sigmoid", "layers": [277, 10, 5, 1],     "epochs": 80,  "lr": 0.002,  "momentum": 0.8},
]

results = []

for i, params in enumerate(hyperparameter_sets, start=1):
    print(f"\n=== Running configuration {i} / {len(hyperparameter_sets)} ===")
    print(params)

    nn, pid_test, y_test, preds, trainer_obj = run_training(
        activation=params["activation"],
        layers=params["layers"],
        epochs=params["epochs"],
        lr=params["lr"],
        momentum=params["momentum"],
        CSV_TRAIN=TRAIN_CSV,
        CSV_VAL=VAL_CSV,
        CSV_TEST=TEST_CSV,
        verbose=False
    )

    metrics = evaluate_model(y_test, preds)

    results.append({
        "layers": params["layers"],
        "epochs": params["epochs"],
        "lr": params["lr"],
        "momentum": params["momentum"],
        "activation": params["activation"],
        "MAPE": metrics["MAPE"],
        "MAE":  metrics["MAE"],
        "MSE":  metrics["MSE"],
    })

df_results = pd.DataFrame(results)
df_results





=== Running configuration 1 / 10 ===
{'activation': 'relu', 'layers': [277, 32, 16, 1], 'epochs': 150, 'lr': 0.001, 'momentum': 0.9}
Training Neural Network...


/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: divide by zero encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: overflow encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: invalid value encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]



=== Running configuration 2 / 10 ===
{'activation': 'relu', 'layers': [277, 64, 32, 1], 'epochs': 200, 'lr': 0.0005, 'momentum': 0.9}
Training Neural Network...


/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: divide by zero encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: overflow encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: invalid value encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:88: RuntimeWarning: divide by zero encountered in matmul
  self.delta[l] = (self.w[l+1].T @ self.delta[l+1]) * self.activate_derivative(self.xi[l])
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evol


=== Running configuration 3 / 10 ===
{'activation': 'tanh', 'layers': [277, 20, 10, 1], 'epochs': 200, 'lr': 0.05, 'momentum': 0.9}
Training Neural Network...


...


=== Running configuration 9 / 10 ===
{'activation': 'tanh', 'layers': [277, 64, 32, 1], 'epochs': 150, 'lr': 0.01, 'momentum': 0.7}
Training Neural Network...


/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: divide by zero encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: overflow encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: invalid value encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:88: RuntimeWarning: divide by zero encountered in matmul
  self.delta[l] = (self.w[l+1].T @ self.delta[l+1]) * self.activate_derivative(self.xi[l])
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evol


=== Running configuration 10 / 10 ===
{'activation': 'sigmoid', 'layers': [277, 10, 5, 1], 'epochs': 80, 'lr': 0.002, 'momentum': 0.8}
Training Neural Network...


/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: divide by zero encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: overflow encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Master/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: invalid value encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]


,layers,epochs,lr,momentum,activation,MAPE,MAE,MSE
0,"[277, 32, 16, 1]",150,0.001000,0.9,relu,28.365423,35956.303279,2.066967e+09
1,"[277, 64, 32, 1]",200,0.000500,0.9,relu,28.295951,35769.213263,2.114754e+09
2,"[277, 20, 10, 1]",200,0.050000,0.9,tanh,13.856358,22306.286786,8.942365e+08
3,"[277, 40, 20, 1]",250,0.020000,0.8,tanh,13.473664,21228.710573,7.739336e+08
4,"[277, 20, 10, 1]",70,0.001000,0.9,sigmoid,29.606848,52375.331070,4.883499e+09
5,"[277, 50, 25, 1]",120,0.005000,0.7,sigmoid,9.753397,18223.772771,6.402909e+08
6,"[277, 1]",200,0.000001,0.0,linear,9.674682,16585.902172,5.206222e+08
7,"[277, 128, 64, 1]",250,0.000300,0.9,relu,28.323504,35911.174612,2.050154e+09
8,"[277, 64, 32, 1]",150,0.010000,0.7,tanh,12.332848,19473.575509,6.797477e+08
9,"[277, 10, 5, 1]",80,0.002000,0.8,sigmoid,23.266009,42505.232285,3.632058e+09
